In [1]:
# Import essentials
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os

In [2]:
train_df = pd.read_excel('train.xlsx') # Load the `train` file

In [3]:
train_df.party[15]


'comuns'

In [5]:
!pip install stop_words

  Running setup.py bdist_wheel for stop-words ... done
  Stored in directory: /home/pablo/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [6]:
import re
from stop_words import get_stop_words

def special_char(text):
    """Retrieve the special characters
    """
    return re.sub(r'\W', ' ', text)

def filter_single(text):
    """remove all single characters
    """
    return re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

def filter_singleStart(text):
    """Remove single characters from the start
    """
    return re.sub(r'\^[a-zA-Z]\s+', ' ', text)

def filter_multiplespace(text):
    """Substituting multiple spaces with single space
    """
    return re.sub(r'\s+', ' ', text, flags=re.I)

def stop_words():
    """Retrieve the stop words for vectorization -Feel free to modify this function
    """
    return get_stop_words('es') + get_stop_words('ca') + get_stop_words('en')

def filter_mentions(text):
    """Utility function to remove the mentions of a tweet
    """
    return re.sub("@\S+", "", text)

def filter_hashtags(text):
    """Utility function to remove the hashtags of a tweet
    """
    return re.sub("#\S+", "", text)

def filter_symb_hashtag(text):
    """Utility function to remove the hashtags symbol of a tweet
    """
    return re.sub("#", "", text)

def filter_symb1(text):
    """Utility function to remove special characters of a tweet
    """
    return re.sub(r'[(:+*?¿!¡.,;-`"")]', "", text)

#def translate_text(text):
#    """Utility function to translate the text of a tweet
#    """
#    return translator.translate(text, dest='es').text

def lower_text(text):
    """Utility function to lower text
    """
    return text.lower()

def eliminate_numbers(text):
    """Utility function to lower text
    """
    return re.sub(r'\w*\d\w*', '', text).strip()

def eliminate_emojiis(text):
    """Utility to eliminate emojiis from the text
    """
    em_pat=re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return  em_pat.sub(r'',text)

### Pre-Processing (1-3 fets)
1) eliminate symbols :,?¿!';-
2) eliminate single characters  
2b) numbers (eliminate numbers and words with numbers)  
3) all to lower  
4) accents? (amb unidecode, pero passa ç a c, ñ a n, aixo introdueix errors?)  
5) i is fessim una columna amb 0-1 indicant si hi ha emoji? (sigui com sigui, pero algu potser els fa servir bastant?)
6) segurament, si hi ha hashtag del partit l'ha posat el mateix partit (publicitat)

In [7]:
train_df['text_clean'] = [filter_symb1(filter_single(eliminate_numbers(lower_text(row['text'])))) for index, row in train_df.iterrows()]
train_df.head(5)

,Id,username,party,text,created_at,retweet_count,favorite_count,text_clean
0,0,martarovira,erc,Últim acte de campanya! Aquí tossudament alçat...,2017-12-19 20:12:01,785,2295,últim acte de campanya aquí tossudament alçats...
1,1,xavierdomenechs,comuns,#Badalona necessita uns pressupostos que posin...,2018-04-27 10:04:19,55,93,#badalona necessita uns pressupostos que posin...
2,2,albert_rivera,cs,Encuentro Villacís-Valls para lanzar una estra...,2018-11-17 20:34:58,357,622,encuentro villacís-valls para lanzar una estra...
3,3,jaumecollboni,psc,"“La palabra es como una bala, no tiene retorno...",2018-10-22 18:10:01,4,6,“la palabra es como una bala no tiene retorno”...
4,4,albiol_xg,ppc,"📻 Esta noche, a partir de las 22:10h, me entre...",2018-08-16 10:30:27,20,47,📻 esta noche partir de las me entrevistan en ...


In [8]:
print (train_df.text[12])
print (train_df.text_clean[12])

“Estas son las reglas del juego en nuestro Estado de Derecho y en nuestra democracia” (Lesmes) la clave aquí está en la palabra "nuestro", "nuestro Estado" 
“estas son las reglas del juego en nuestro estado de derecho en nuestra democracia” lesmes la clave aquí está en la palabra nuestro nuestro estado


### Afegim una columna amb idioma
tot i que alguns resultats ens han mostrat pitjors scores

In [10]:
# Translate text to spanish and define column (vector) with language (spa:0, cat:1)
!pip install langdetect
from langdetect import detect 
train_df['language'] = [detect(row['text']) for index, row in train_df.iterrows()]

# language column to 1-0 --> esta columna lang_val la ponemos al final del array (Xvec)
train_df['lang_val'] = np.where(train_df.language == 'ca', 1, 0)
train_df.head(5)



    100% |████████████████████████████████| 1.0MB 1.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for langdetect ... done
  Stored in directory: /home/pablo/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


,Id,username,party,text,created_at,retweet_count,favorite_count,text_clean,language,lang_val
0,0,martarovira,erc,Últim acte de campanya! Aquí tossudament alçat...,2017-12-19 20:12:01,785,2295,últim acte de campanya aquí tossudament alçats...,ca,1
1,1,xavierdomenechs,comuns,#Badalona necessita uns pressupostos que posin...,2018-04-27 10:04:19,55,93,#badalona necessita uns pressupostos que posin...,ca,1
2,2,albert_rivera,cs,Encuentro Villacís-Valls para lanzar una estra...,2018-11-17 20:34:58,357,622,encuentro villacís-valls para lanzar una estra...,es,0
3,3,jaumecollboni,psc,"“La palabra es como una bala, no tiene retorno...",2018-10-22 18:10:01,4,6,“la palabra es como una bala no tiene retorno”...,es,0
4,4,albiol_xg,ppc,"📻 Esta noche, a partir de las 22:10h, me entre...",2018-08-16 10:30:27,20,47,📻 esta noche partir de las me entrevistan en ...,es,0


### coses per fer:
- lemmatization: sino ho podem traduir (el googletranslator dona problemes) no el podem aplicar (no n'hi ha en catala)  
- identificar hastags? ara per ara han quedat

# Models
Ho fem fent servir el pipeline (aixo ens permet aplicar una funcio per buscar parametres optims)

## Linear SVC
### Resultats
1. pipeline per trobar millors parametres:  
score = 0.72 -->  Text Clean  
{'clf__C': 1.0, 'clf__dual': False, 'clf__fit_intercept': True, 'clf__intercept_scaling': 1, 'clf__penalty': 'l2', 'vect__max_features': 6000, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__stop_words':
2. apliquem model sobre variacions del vector? per exemple treient hasthtags? 
3.  

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, LinearSVR, SVC
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', LinearSVC()),
])

X_train, X_test, y_train, y_test = train_test_split(train_df['text_clean'], train_df['party'].values, test_size=0.20, random_state=345 )
param_grid = [
    { 
          "clf__penalty": ["l2"],
          "clf__dual":[False,True],
          "clf__C":[0.1, 1.0],
          "clf__fit_intercept":[True,False],
          "clf__intercept_scaling":[0.1,1,10],
          "vect__min_df":[1,2,3,4],
          "vect__ngram_range": [(1, 1), (1, 2),(1, 3)],
          "vect__stop_words":[stop_words()],
          "vect__max_features":[6000, 8000, 10000]
    },
    { 
          "clf__penalty": ["l1"],
          "clf__dual":[False],
          "clf__C":[0.1, 1.0],
          "clf__fit_intercept":[True,False],
          "clf__intercept_scaling":[0.1,1,10],
          "vect__min_df":[1,2,3,4],
          "vect__ngram_range": [(1, 1),(1, 2),(1, 3)],
          "vect__stop_words":[stop_words()],
          "vect__max_features":[6000, 8000, 10000]
    }
]

grid = GridSearchCV(pipeline, cv=6, param_grid=param_grid)
grid.fit(X_train,y_train)


print ("score = %3.2f" %(grid.score(X_test,y_test)) )
print (pd.DataFrame.from_dict(grid.cv_results_).loc[grid.best_index_ ])
print (grid.best_params_ )

/home/pablo/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'des', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/pablo/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


KeyboardInterrupt: 

## Logistic Regression
### Resultats
1.  score = 0.72
{'clf__C': 10, 'clf__dual': True, 'clf__fit_intercept': True, 'clf__penalty': 'l2', 'clf__solver': 'liblinear', 'clf__tol': 0.0001, 'vect__max_features': 6000, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__stop_words':
2.  
3.  

In [105]:
from sklearn.linear_model import LogisticRegression
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', LogisticRegression()),
])

X_train, X_test, y_train, y_test = train_test_split(train_df['text_clean'], train_df['party'].values, test_size=0.20, random_state=345 )
param_grid = [
    { 
          "clf__penalty": ["l2"],
          "clf__dual":[True],
          "clf__solver":['liblinear'],
          "clf__tol":[0.0001, 0.001, 0.01],
          "clf__C":[0.1,1,10],
          "clf__fit_intercept":[True, False],
          "vect__min_df":[1,2],
          "vect__ngram_range": [(1, 1), (1, 2),(1, 3)],
          "vect__stop_words":[stop_words()],
          "vect__max_features":[6000, 8000]
    },
    { 
          "clf__penalty": ["l2"],
          "clf__dual":[False],
          "clf__solver":['lbfgs','newton-cg','sag'],
          "clf__tol":[0.0001, 0.001, 0.01],
          "clf__C":[0.1,1,10],
          "clf__fit_intercept":[True, False],
          "vect__min_df":[1,2],
          "vect__ngram_range": [(1, 1), (1, 2),(1, 3)],
          "vect__stop_words":[stop_words()],
          "vect__max_features":[6000, 8000]
    },
    { 
          "clf__penalty": ["l1"],
          "clf__dual":[False],
          "clf__solver":['liblinear', 'saga'],
          "clf__tol":[0.0001, 0.001, 0.01],
          "clf__C":[0.1,1,10],
          "clf__fit_intercept":[True, False],
          "vect__min_df":[1,2],
          "vect__ngram_range": [(1, 1), (1, 2),(1, 3)],
          "vect__stop_words":[stop_words()],
          "vect__max_features":[6000, 8000]
    }
]
grid = GridSearchCV(pipeline, cv=6, param_grid=param_grid)
grid.fit(X_train,y_train)


print ("score = %3.2f" %(grid.score(X_test,y_test)) )
print (grid.best_params_ )

C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

...

C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

score = 0.72
{'clf__C': 10, 'clf__dual': True, 'clf__fit_intercept': True, 'clf__penalty': 'l2', 'clf__solver': 'liblinear', 'clf__tol': 0.0001, 'vect__max_features': 6000, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__stop_words': ['a', 'al', 'algo', 'algunas', 'algunos', 'ante', 'antes', 'como', 'con', 'contra', 'cual', 'cuando', 'de', 'del', 'desde', 'donde', 'durante', 'e', 'el', 'ella', 'ellas', 'ellos', 'en', 'entre', 'era', 'erais', 'eran', 'eras', 'eres', 'es', 'esa', 'esas', 'ese', 'eso', 'esos', 'esta', 'estaba', 'estabais', 'estaban', 'estabas', 'estad', 'estada', 'estadas', 'estado', 'estados', 'estamos', 'estando', 'estar', 'estaremos', 'estará', 'estarán', 'estarás', 'estaré', 'estaréis', 'estaría', 'estaríais', 'estaríamos', 'estarían', 'estarías', 'estas', 'este', 'estemos', 'esto', 'estos', 'estoy', 'estuve', 'estuviera', 'estuvierais', 'estuvieran', 'estuvieras', 'estuvieron', 'estuviese', 'estuvieseis', 'estuviesen', 'estuvieses', 'estuvimos', 'estuviste', '

In [106]:
print ("score = %3.2f" %(grid.score(X_test,y_test)) )
print (pd.DataFrame.from_dict(grid.cv_results_).loc[grid.best_index_ ])
print (grid.best_params_ )

score = 0.72
mean_fit_time                                                        0.119832
std_fit_time                                                       0.00134786
mean_score_time                                                     0.0123274
std_score_time                                                     0.00093277
param_clf__C                                                               10
param_clf__dual                                                          True
param_clf__fit_intercept                                                 True
param_clf__penalty                                                         l2
param_clf__solver                                                   liblinear
param_clf__tol                                                         0.0001
param_vect__max_features                                                 6000
param_vect__min_df                                                          1
param_vect__ngram_range                            

C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Pablo\Anaconda3\lib\site-packages\sklearn\util

# SGD Classifier
## Results
1.  score = 0.71
{'clf__alpha': 1e-05, 'clf__loss': 'hinge', 'clf__max_iter': 80, 'clf__penalty': 'l2', 'vect__max_features': 8000, 'vect__min_df': 1, 'vect__ngram_range': (1, 1)}
2.  
3.  


In [108]:
from sklearn.linear_model import SGDClassifier

pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', SGDClassifier()),
])

X_train, X_test, y_train, y_test = train_test_split(train_df['text_clean'], train_df['party'].values, test_size=0.20, random_state=345 )

#clf = SGDClassifier(loss='log', penalty='l1',alpha=1e-6, max_iter=50,random_state=42).fit(X_train, y_train)

parameters = {
    'vect__min_df': (1, 2),
    'vect__max_features': (6000, 8000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    # 'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
    'clf__loss': ('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'),
    'clf__alpha': (0.0001, 0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    'clf__max_iter': (10, 50, 80),
}

grid = GridSearchCV(pipeline, cv=6, param_grid=parameters)
grid.fit(X_train,y_train)

print ("score = %3.2f" %(grid.score(X_test,y_test)) )
print (grid.best_params_ )

prediction_train = grid.predict(X_train)
prediction = grid.predict(X_test)

print(confusion_matrix(y_test,prediction))  
print(classification_report(y_test,prediction))  
print(accuracy_score(y_test, prediction))  

score = 0.71
{'clf__alpha': 1e-05, 'clf__loss': 'hinge', 'clf__max_iter': 80, 'clf__penalty': 'l2', 'vect__max_features': 8000, 'vect__min_df': 1, 'vect__ngram_range': (1, 1)}


NameError: name 'confusion_matrix' is not defined